# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 0: Set up connection for use in corporate network

In [1]:
# Import self-defined library to be able to use sensitive data
import sensitivedata
sensitivedata.corporate_environment_connection()

## Part 1: Create the dataframe (scrape Wikipedia, pre-process data as requested)

## Import libraries

In [2]:
# Import libraries
import pandas as pd
import numpy as np

## Scrape wikipedia

In [3]:
# Scrape wikipedia website to retrieve the original table
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# only use the first table for this analysis
df = df[0]

## Preprocess data as requested in assignment

In [4]:
# Drop rows where Borough is 'Not assigned' or NaN
df = df.drop(df[df['Borough']=='Not assigned'].index)
df = df.drop(df[df['Borough']==np.NaN].index)
# If Neihbourhood is 'Not assigned' or NaN, then replace with  value from Borough
df.replace(to_replace=(df['Neighbourhood']=='Not assigned'), value=df['Borough'],inplace=True)
df.replace(to_replace=(df['Neighbourhood']==np.NaN), value=df['Borough'],inplace=True)

In [5]:
# Merge neighbourhoods for all postal code / borough comibnation
df = df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df = df.drop_duplicates()

## Show a few samples and the shape

In [6]:
# show some examples, NOTE: Neighbourhood contains the data as requested if several neighbourhoods are asssociated with a borrow
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# Show the shape of the dataframe
df.shape

(103, 3)

## Part 2: Get location information

### Note: The geocoder library didn't return results despite many tries, hence I will use the alternative as proposed in the instructions. I leave the code for geocoder library for demonstration purposes.

### 2A) Use of Geocoder library - commented out due to retrieval issues with the library / API

In [8]:
# Install geocoder library
# pip install geocoder

In [9]:
# Import geocoder library
# import geocoder

In [10]:
# Function to retrieve location for a given postcode

# def getlocation(postal_code):
#    # initialize your variable to None
#    lat_lng_coords = None
#
#    # loop until you get the coordinates 
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng

#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]

#    return latitude, longitude

In [11]:
# Create columns for latitude and logitude 
# df.insert(loc=3,column='Latitude',value=np.NaN)
# df.insert(loc=4,column='Longitude',value=np.NaN)

# Get locations for all postcodes
# df['Latitude'], df['Longitude'] = getlocation(df['Postal Code'])

### 2B) Retrieve location information via alternative CSV file as proposed in instructions

In [12]:
# Retrieve CSV file
df_latlong = pd.read_csv(r"C:\Users\A2712835\AppData\Local\WPy64-3800\notebooks\data\Geospatial_Coordinates.csv")
# Merge dataframes based on postal code
df = pd.merge(df, df_latlong, on="Postal Code")

In [13]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df.shape

(103, 5)

## Part 3: Segment and cluster boroughs that contain Toronto in their borough name

### To be able to view the maps, please use the NBViewer display of this notebook at
### <a href='https://nbviewer.jupyter.org/github/rolandteschke/Coursera_Capstone/blob/main/Segment%20and%20Cluster%20Neighborhoods%20in%20Toronto.ipynb'> https://nbviewer.jupyter.org/github/rolandteschke/Coursera_Capstone/blob/main/Segment%20and%20Cluster%20Neighborhoods%20in%20Toronto.ipynb </href>
### <br> Import libraries

In [15]:
# Import additional libraries

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

### Slice dataframe and select data to be processed further

In [16]:
# Slice datframe to only select boroughs that contain 'Toronto'
toronto_data = df[df['Borough'].str.contains('Toronto')]
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
# Display the shape of the sliced dataframe
toronto_data.shape

(39, 5)

### Show selected neighbourhoods on a map, still without clustering at this stage

In [18]:
# Get location for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add neigbourhood markers to Toronto map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
# Show Toronto map first with the selected neighbourhood areas    
toronto_map

### Get venue information for selected neighborhoods via Foursquare

In [19]:
# Get Foursquare credentials from my self-defined class so that my credentials are kept secret
CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN = sensitivedata.my_foursquare_credentials()

In [20]:
VERSION = '20200127'
LIMIT = '100'
# Function to request max. 100 venues within 500m radius the selected neighbourhoods centers
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('Retrieval complete')
    return(nearby_venues)

In [21]:
# Retrieve venues
toronto_venues = getNearbyVenues(toronto_data['Neighbourhood'], toronto_data['Latitude'], toronto_data['Longitude'])

Retrieval complete


In [22]:
# quickly inspect venues retrieved
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [23]:
# Display shape of new dataframe
toronto_venues.shape

(1608, 7)

In [24]:
# Show some examples how many venues in a specific neighbourhood (output limited to 10 rows for better legibility)
toronto_venues.groupby('Neighbourhood').count().head(10)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


### Apply one hot encoding and data preprocessing to prepare data for clustering analysis in subsequent steps

In [25]:
# one hot encoding to prepare dataset for KMeans clustering
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_onehot.shape

(1608, 236)

In [27]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.016129


In [28]:
#Show shape f grouped dataframe
toronto_grouped.shape

(39, 236)

In [29]:
# Sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
# Create dataframe and display Top 10 venues by neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Clothing Store,Lounge
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Burrito Place,Intersection,Bar,Italian Restaurant,Bakery,Restaurant,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Office,Juice Bar,Korean Restaurant


In [31]:
# Drop neighboordhood name for cluster analysis
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

### Perform a cluster analysis via KMeans algorithms. <br><br> I apply a grid search in order to identify the optimum cluster number, using the standard KMeans scoring metric to optimize for. I search in the range between 3 and 15 clusters. I output the results from the grid search for a sanity check. Afterwards I keep processing the model that yielded the best result.

In [32]:
# Use grid search to optimize number of clusters for kmeans clustering analysis

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# Define Search Parameters
kclusters = np.arange(3,15,1).tolist() # set a range of 3 to 15 clusters to perform a grid search to discover optimum number of clusters
search_params = {'n_clusters': kclusters, # range of cluster sizes to iterate
                 'random_state':[42] # ensure results are going to be repeatable in future executions
                }

# Init the Model
kmeans = KMeans()

# Init Grid Search Class
model = GridSearchCV(kmeans, param_grid=search_params)

# Do the Grid Search
model.fit(toronto_grouped_clustering)

# Best Model and set value of labels / clusters for use in map display
best_kmeans_model = model.best_estimator_
kclusters = model.best_params_['n_clusters']

pd.set_option('display.max_columns', None)
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').head()

C:\Users\A2712835\AppData\Local\WPy64-3800\python-3.8.0.amd64\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_clusters,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
4,0.024482,0.000810,0.000694,0.000233,7,42,"{'n_clusters': 7, 'random_state': 42}",-0.861358,-1.014283,-1.199469,-1.025037,0.138242,1
6,0.046501,0.003084,0.001000,0.000002,9,42,"{'n_clusters': 9, 'random_state': 42}",-0.862617,-1.003300,-1.225588,-1.030502,0.149426,2
7,0.047835,0.004189,0.001167,0.000236,10,42,"{'n_clusters': 10, 'random_state': 42}",-0.858364,-1.030526,-1.214645,-1.034512,0.145479,3
5,0.036650,0.007519,0.001168,0.000471,8,42,"{'n_clusters': 8, 'random_state': 42}",-0.876199,-1.017651,-1.211866,-1.035239,0.137599,4
1,0.016349,0.000247,0.000677,0.000239,4,42,"{'n_clusters': 4, 'random_state': 42}",-0.874426,-1.011757,-1.230128,-1.038770,0.146465,5


In [33]:
# check cluster labels generated for each row in the dataframe
best_kmeans_model.labels_ 

array([0, 0, 0, 0, 0, 6, 0, 0, 6, 0, 6, 0, 1, 0, 0, 6, 6, 0, 5, 0, 3, 0,
       0, 0, 0, 0, 4, 2, 0, 0, 6, 0, 0, 0, 6, 6, 6, 0, 6])

### Adding cluster labels based on the optimum model identified

In [34]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', best_kmeans_model.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Park,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Discount Store,Department Store,Dessert Shop,Diner
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,6,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Pub
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,6,Pizza Place,Park,Fast Food Restaurant,Pub,Fish & Chips Shop,Sushi Restaurant,Light Rail Station,Restaurant,Italian Restaurant,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Park,Business Service,Swim School,Bus Line,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Show the Toronto map, now with clustered neighbourhoods

In [35]:
# Show the clustered map of neighbourhoods that contain Toronto in their name

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Show more details for the identified clusters, here grouped by clusters (one could theoretically alternatively add neighbourhoods as second grouping parameter if needed; kept short here using only one groupby parameter for better legibility)

In [36]:
# See data to get a feeling what the clusters represent - show top
cluster_details = toronto_merged.sort_values('Cluster Labels').groupby(by=('Cluster Labels'))
cluster_details.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Italian Restaurant,Pizza Place
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Coffee Shop,Café,Sushi Restaurant,Pub,Italian Restaurant,Electronics Store,Bar,Indie Movie Theater,Fish & Chips Shop,Smoothie Shop
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Breakfast Spot,Gift Shop,Restaurant,Bar,Dessert Shop,Eastern European Restaurant,Coffee Shop,Bookstore,Italian Restaurant,Movie Theater
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,0,Café,Breakfast Spot,Coffee Shop,Burrito Place,Intersection,Bar,Italian Restaurant,Bakery,Restaurant,Climbing Gym
77,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Restaurant,Bakery,Café,Men's Store,Asian Restaurant,Cuban Restaurant,Brewery
64,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Music Venue,Home Service,Garden,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Lawyer,Playground,Tennis Court,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Park,Business Service,Swim School,Bus Line,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
